In [10]:
import sys
import pandas as pd
import numpy as np
sys.path.append("d:\\BaiduNetdiskWorkspace\\2-code\\FundResearch\\")

import quant_utils.data_moudle as dm
from quant_utils.constant import DB_CONN_LOCAL_MYSQL

In [11]:
portfolio_holding = pd.read_excel("D:/BaiduNetdiskWorkspace/1-基金投研/5-data/组合配置权重.xlsx")
portfolio_holding['产品代码'] = portfolio_holding['产品代码'].apply(lambda s: str(s).rjust(6, '0'))
start_date = '20220705'
end_date = '20221031'

port_list = []
for portfolio_name in portfolio_holding['组合名称'].unique():
    print(portfolio_name)
    port_weights = portfolio_holding[portfolio_holding['组合名称'] == portfolio_name][['产品代码', '开始权重']]
    port_weights = port_weights.sort_values(by='产品代码')
    port_weights = port_weights.set_index('产品代码')

    # 提取净值
    result_list = []
    for ticker in port_weights.index:
        temp = dm.get_fund_adj_nav(ticker_symbol=ticker, start_date=start_date, end_date=end_date)
        if temp.empty:
            temp = dm.get_cash_fund_adj_nav(ticker_symbol=ticker, start_date=start_date, end_date=end_date)
        temp['ret'] = temp['ADJUST_NAV'].pct_change().dropna()
        result_list.append(temp)

    ret_df = pd.concat(result_list)
    # 收益率
    ret_df = ret_df.pivot_table(columns='TICKER_SYMBOL', index='TRADE_DT', values='ret')

    # 协方差矩阵
    print(port_weights.index)
    cov_matrix = ret_df.cov()*252
    cov_array =  np.asarray(cov_matrix).dot(port_weights.values)
    port_std = np.sqrt(port_weights.values.T.dot(cov_array))
    # 边际风险贡献
    margin_risk_contribution = cov_array/port_std
    # 风险贡献
    risk_contribution = port_weights.values * margin_risk_contribution

    risk_contribution = pd.DataFrame(risk_contribution, columns=['风险贡献'])
    risk_contribution["风险占比"] = risk_contribution['风险贡献']/port_std[0]
    risk_contribution['组合名称'] = portfolio_name
    risk_contribution['产品代码'] = port_weights.index
    risk_contribution['边际风险贡献'] = margin_risk_contribution
    risk_contribution = risk_contribution[['组合名称', '产品代码', '边际风险贡献', '风险贡献', '风险占比']]
    port_list.append(risk_contribution)

port_risk = pd.concat(port_list)
portfolio_holding = portfolio_holding.merge(port_risk)

月月享
Index(['003328', '004920', '006647', '007016', '100018', '100058', '110017',
       '163415', '166005', '233005', '340005', '519697', '519718', '519783',
       '531021'],
      dtype='object', name='产品代码')
进取全明星
Index(['000083', '000263', '000566', '000831', '006608', '007016', '007143',
       '007802', '110012', '163415', '166005', '166009', '260101', '340005',
       '519697'],
      dtype='object', name='产品代码')
固收佳进取270天
Index(['001820', '003328', '004920', '007016', '007075', '007100', '010339',
       '110012', '110018', '485011', '519697', '519720', '519783', '531021'],
      dtype='object', name='产品代码')
均衡小确幸
Index(['000171', '000216', '000263', '000566', '005852', '006608', '007016',
       '007143', '100018', '110012', '110017', '163415', '166005', '233005',
       '340005', '485111', '519697'],
      dtype='object', name='产品代码')


In [12]:
portfolio_holding

,组合名称,产品代码,产品名称,开始权重,区间总收益率,收益贡献度,收益率*权重,结束权重,边际风险贡献,风险贡献,风险占比
0,月月享,003328,万家鑫璟纯债C,0.0888,0.0175,0.1334,0.0016,0.0914,0.002021,1.794690e-04,0.008638
1,月月享,004920,富国泓利纯债债券型发起式A,0.0791,0.0128,0.0870,0.0010,0.0811,0.000249,1.970692e-05,0.000949
2,月月享,006647,添富短债债券C,0.0593,0.0060,0.0305,0.0004,0.0603,0.000207,1.226624e-05,0.000590
3,月月享,007016,富国睿泽回报混合,0.0267,-0.1521,-0.3482,-0.0041,0.0229,0.094449,2.521784e-03,0.121382
4,月月享,100018,富国天利增长债券,0.0989,0.0117,0.0994,0.0012,0.1013,0.004981,4.926028e-04,0.023711
...,...,...,...,...,...,...,...,...,...,...,...
56,均衡小确幸,166005,中欧价值发现混合A,0.0506,-0.1088,-0.0832,-0.0055,0.0483,0.146532,7.414495e-03,0.092309
57,均衡小确幸,233005,大摩强收益债券,0.0869,-0.0182,-0.0239,-0.0016,0.0913,0.017638,1.532749e-03,0.019082
58,均衡小确幸,340005,兴全货币A,0.0190,0.0057,0.0016,0.0001,0.0205,-0.000021,-3.984452e-07,-0.000005
59,均衡小确幸,485111,工银双利债券A,0.0484,0.0195,0.0143,0.0009,0.0529,0.014296,6.919496e-04,0.008615


In [13]:
portfolio_holding.to_excel('D:/BaiduNetdiskWorkspace/1-基金投研/5-data/风险占比.xlsx')

In [16]:
sql_query = """
SELECT
	a.TICKER_SYMBOL,
	a.F_INFO_NAME,
	a.LEVEL_1,
	a.LEVEL_2,
	b.INDUSTRY_NAME,
	b.MARKET_VALUE,
	b.INDUSTRY_RATIO_IN_NA
FROM
	fund_type_own a
  JOIN fund_holding_industry b ON b.TICKER_SYMBOL = a.TICKER_SYMBOL and a.REPORT_DATE = b.REPORT_DATE
WHERE
	a.REPORT_DATE = '20220630'
"""
industry = DB_CONN_LOCAL_MYSQL.exec_query(sql_query)

In [17]:
port_name = '全明星调整后'

market_status = (industry.query("LEVEL_1 == '主动权益'").groupby(by='INDUSTRY_NAME')['MARKET_VALUE'].sum())/(industry.query("LEVEL_1 == '主动权益'")['MARKET_VALUE'].sum())*100
port_df = portfolio_holding[portfolio_holding['组合名称']==port_name].merge(industry, right_on='TICKER_SYMBOL', left_on='产品代码')
port_df['weighted_mv'] = port_df['开始权重'] * port_df['INDUSTRY_RATIO_IN_NA']
port_industry = port_df.groupby(by='INDUSTRY_NAME')['weighted_mv'].sum()/port_df['weighted_mv'].sum()*100
port_industry = pd.DataFrame(port_industry).merge(pd.DataFrame(market_status), how='outer', left_index=True, right_index=True)
port_industry['差值'] = port_industry['weighted_mv'].fillna(0) - port_industry['MARKET_VALUE'].fillna(0)
port_industry.columns = [port_name, '偏股基金', '差值']
port_industry.sort_values(by='偏股基金', ascending=False, inplace=True)

In [18]:
port_df.pivot_table(columns='产品代码', index='INDUSTRY_NAME', values='INDUSTRY_RATIO_IN_NA').fillna(0)

产品代码,000083,000263,000566,000831,001980,007016,007126,007143,007802,040015,110012,161222,166009,519697
INDUSTRY_NAME,,,,,,,,,,,,,,
交通运输,0.00,0.00,8.57,0.00,3.37,0.00,1.59,2.16,0.00,3.24,2.70,0.00,7.80,0.00
传媒,0.00,0.00,0.00,0.00,0.70,0.00,0.60,0.65,1.66,0.21,0.00,0.00,0.00,0.00
公用事业,0.00,0.00,0.02,0.00,1.52,0.00,1.80,1.62,0.68,0.93,0.64,3.91,0.00,0.00
农林牧渔,0.00,1.89,6.42,0.00,1.78,0.00,1.17,0.35,0.00,0.00,7.37,6.91,6.30,0.00
医药生物,7.89,5.05,9.56,82.43,9.14,5.04,5.69,7.13,0.16,2.87,4.31,6.28,0.08,28.85
商贸零售,5.40,0.00,1.08,0.00,0.68,0.00,0.48,1.04,3.73,1.13,0.00,0.00,1.26,0.00
国防军工,0.00,8.32,8.10,0.00,2.62,5.21,1.95,1.18,6.84,20.72,2.04,3.16,0.00,0.03
基础化工,0.00,1.93,6.11,0.00,9.18,5.09,10.98,2.62,2.41,6.41,11.48,0.00,6.91,8.94
家用电器,6.50,0.00,3.60,0.00,2.35,1.16,1.62,2.68,2.98,0.00,3.58,1.13,1.89,0.00
